In [164]:
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools


In [165]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/danc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [166]:
import string
import re
# DataFrame
import pandas as pd
from nltk.corpus import stopwords 
# stopwords_english = stopwords.words('english')
stop_words = stopwords.words("english")

In [167]:
cols = ['sentiment','id','date','query_string','user','text']
df_test = pd.read_csv("/Users/danc/Downloads/labelled data/std140/testdata.manual.2009.06.14.csv",encoding = "ISO-8859-1", header=None, names=cols)

In [168]:
df_test.head()

,sentiment,id,date,query_string,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [169]:
stop_words = stopwords.words("english")
# stemmer = SnowballStemmer("english")

In [170]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()    
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [171]:
df_test['cleaned_text'] = df_test.text.apply(lambda x: preprocess(x))

In [172]:
df_test.head()

,sentiment,id,date,query_string,user,text,cleaned_text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,loooooooovvvvvveee kindle2 dx cool 2 fantastic...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,reading kindle2 love lee childs good read
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",ok first assesment kindle2 fucking rocks
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,love kindle2 mine months never looked back new...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,fair enough kindle2 think perfect


In [173]:
# df.drop(['id','date','query_string'],axis=1,inplace=True)

In [174]:
all_reviews = df_test['cleaned_text']
all_sent_values = []
all_sentiments = []

In [175]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentiment_value(paragraph):
    analyser = SentimentIntensityAnalyzer()
    result = analyser.polarity_scores(paragraph)
    return result
#     score = result['compound']
#     return round(score,3)

In [176]:
sample = df_test['cleaned_text'][456]
print(sample)
print('Sentiment: ')
print(sentiment_value(sample))

love kindle2 stacks books trip way loo
Sentiment: 
{'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.6369}


In [177]:
len(df_test)

498

In [178]:
for i in range(0,len(df_test)):
    all_sent_values.append(sentiment_value(all_reviews[i]))

In [179]:
df_test['all_dict'] = all_sent_values

In [180]:
df_test.head()

,sentiment,id,date,query_string,user,text,cleaned_text,all_dict
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,loooooooovvvvvveee kindle2 dx cool 2 fantastic...,"{'neg': 0.31, 'neu': 0.233, 'pos': 0.457, 'com..."
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,reading kindle2 love lee childs good read,"{'neg': 0.0, 'neu': 0.413, 'pos': 0.587, 'comp..."
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",ok first assesment kindle2 fucking rocks,"{'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'comp..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,love kindle2 mine months never looked back new...,"{'neg': 0.113, 'neu': 0.538, 'pos': 0.349, 'co..."
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,fair enough kindle2 think perfect,"{'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'comp..."


In [181]:
# SENTIMENT_VALUE = []
# SENTIMENT = []
# for i in range(0,len(df_test)):
#     sent = all_sent_values[i]['compound']
#     if (sent<=1 and sent>=0.5):
#         SENTIMENT.append('V.Positive')
#         SENTIMENT_VALUE.append(5)
#     elif (sent<0.5 and sent>0):
#         SENTIMENT.append('Positive')
#         SENTIMENT_VALUE.append(4)
#     elif (sent==0):
#         SENTIMENT.append('Neutral')
#         SENTIMENT_VALUE.append(3)
#     elif (sent<0 and sent>=-0.5):
#         SENTIMENT.append('Negative')
#         SENTIMENT_VALUE.append(2)
#     else:
#         SENTIMENT.append('V.Negative')
#         SENTIMENT_VALUE.append(1)
        

In [182]:
SENTIMENT_VALUE = []
SENTIMENT = []
for i in range(0,len(df_test)):
    sent = all_sent_values[i]['compound']
    if (sent<=1 and sent>0):
        SENTIMENT.append('Positive')
        SENTIMENT_VALUE.append(4)
    elif (sent==0):
        SENTIMENT.append('Neutral')
        SENTIMENT_VALUE.append(2)
    else:
        SENTIMENT.append('Negative')
        SENTIMENT_VALUE.append(0)
        

In [183]:
# all_sent_values[4]['compound']

In [184]:
df_test['SENTIMENT_VALUE'] = SENTIMENT_VALUE
df_test['SENTIMENT'] = SENTIMENT

In [185]:
df_test

,sentiment,id,date,query_string,user,text,cleaned_text,all_dict,SENTIMENT_VALUE,SENTIMENT
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...,loooooooovvvvvveee kindle2 dx cool 2 fantastic...,"{'neg': 0.31, 'neu': 0.233, 'pos': 0.457, 'com...",4,Positive
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...,reading kindle2 love lee childs good read,"{'neg': 0.0, 'neu': 0.413, 'pos': 0.587, 'comp...",4,Positive
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck...",ok first assesment kindle2 fucking rocks,"{'neg': 0.0, 'neu': 0.694, 'pos': 0.306, 'comp...",4,Positive
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...,love kindle2 mine months never looked back new...,"{'neg': 0.113, 'neu': 0.538, 'pos': 0.349, 'co...",4,Positive
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...,fair enough kindle2 think perfect,"{'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'comp...",4,Positive
5,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...,big quite happy kindle2,"{'neg': 0.0, 'neu': 0.429, 'pos': 0.571, 'comp...",4,Positive
6,0,9,Mon May 11 03:22:30 UTC 2009,aig,Seth937,Fuck this economy. I hate aig and their non lo...,fuck economy hate aig non loan given asses,"{'neg': 0.545, 'neu': 0.455, 'pos': 0.0, 'comp...",0,Negative
7,4,10,Mon May 11 03:26:10 UTC 2009,jquery,dcostalis,Jquery is my new best friend.,jquery new best friend,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",4,Positive
8,4,11,Mon May 11 03:27:15 UTC 2009,twitter,PJ_King,Loves twitter,loves twitter,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",4,Positive
9,4,12,Mon May 11 03:29:20 UTC 2009,obama,mandanicole,how can you not love Obama? he makes jokes abo...,love obama makes jokes,"{'neg': 0.0, 'neu': 0.244, 'pos': 0.756, 'comp...",4,Positive


In [186]:
#find accuracy
counter = 0
for i in range(0,len(df_test)):
    if (abs(df_test['sentiment'][i]-df_test['SENTIMENT_VALUE'][i])>1):
        counter += 1
    

In [187]:
accuracy = (df_test.shape[0]-counter)/df_test.shape[0]
percent_accuracy = accuracy*100

In [188]:
percent_accuracy

69.27710843373494

In [189]:
# https://github.com/cjhutto/vaderSentiment

In [190]:
SENTIMENT_VALUE = []
SENTIMENT = []
for i in range(0,len(df_test)):
    sent = all_sent_values[i]['compound']
    if (sent>=0.5):
        SENTIMENT.append('Positive')
        SENTIMENT_VALUE.append(4)
    elif (sent>-0.5 and sent<0.5):
        SENTIMENT.append('Neutral')
        SENTIMENT_VALUE.append(2)
    else:
        SENTIMENT.append('Negative')
        SENTIMENT_VALUE.append(0)
        

In [191]:
df_test['SENTIMENT_VALUE'] = SENTIMENT_VALUE
df_test['SENTIMENT'] = SENTIMENT

In [192]:
#find accuracy
counter = 0
for i in range(0,len(df_test)):
    if (abs(df_test['sentiment'][i]-df_test['SENTIMENT_VALUE'][i])>1):
        counter += 1
    

In [193]:
accuracy = (df_test.shape[0]-counter)/df_test.shape[0]
percent_accuracy = accuracy*100
percent_accuracy

57.831325301204814